# RDBMS

In [ ]:
import sqlite3
#connect to the database
conn = sqlite3.connect("mydb.db")
cursor = conn.cursor()
#.db format to file khol ra hernu chai "db Browser for Sqlite" bhanne soft ware download garnu parxa

In [2]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS students (
               id INTEGER PRIMARY KEY,
               name TEXT,
               age INTEGER,
               address TEXT)
""")

In [4]:
cursor.execute('INSERT into students (name, age, address) VALUES (?,?,?)',
               ("manav", 20, "dharan"))
conn.commit()

In [8]:
cursor.execute('INSERT into students (name, age, address) VALUES (?,?,?)',
               ("mausam", 20, "kerabari"))
cursor.execute('INSERT into students (name, age, address) VALUES (?,?,?)',
               ("himal", 20, "jhapa"))
conn.commit()


In [10]:
cursor.execute('SELECT * FROM students')
rows = cursor.fetchall()
print (rows)


[(1, 'manav', 20, 'dharan'), (2, 'manav', 20, 'dharan'), (3, 'mausam', 20, 'kerabari'), (4, 'himal', 20, 'jhapa'), (5, 'mausam', 20, 'kerabari'), (6, 'himal', 20, 'jhapa')]


In [13]:
cursor.execute('SELECT * FROM students WHERE address ="dharan"')
row = cursor.fetchall()
print (row)


[(1, 'manav', 20, 'dharan'), (2, 'manav', 20, 'dharan')]


In [ ]:
cursor.execute('SELECT * FROM students WHERE name LIKE "m%"')     #'m' bata start hune sab name dinxa
# cursor.execute('SELECT * FROM students WHERE name = "%M"')      #'m' bata end hune sab names haru dinxa
m_bata_start_hune = cursor.fetchall()
print (m_bata_start_hune)

[(1, 'manav', 20, 'dharan'), (2, 'manav', 20, 'dharan'), (3, 'mausam', 20, 'kerabari'), (5, 'mausam', 20, 'kerabari')]


# Using ORM

In [ ]:
#step 1: engine banaune
from sqlalchemy import create_engine
engine = create_engine('sqlite:///anotherdb.db', echo = True)


In [19]:
#step2: making of the table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime

base = declarative_base()


class Student(base):
    __tablename__ = 'students'    #k nam ko table banaune bhanera yesma lekhne
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    address = Column(String)

#Now, creating a table

base.metadata.create_all(engine)

2025-12-31 13:13:28,739 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-31 13:13:28,741 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2025-12-31 13:13:28,742 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-31 13:13:28,745 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("students")
2025-12-31 13:13:28,746 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-31 13:13:28,749 INFO sqlalchemy.engine.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	age INTEGER, 
	address VARCHAR, 
	PRIMARY KEY (id)
)


2025-12-31 13:13:28,751 INFO sqlalchemy.engine.Engine [no key 0.00196s] ()
2025-12-31 13:13:28,757 INFO sqlalchemy.engine.Engine COMMIT


/tmp/ipykernel_9854/744977665.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  base = declarative_base()


In [20]:
#step 3: creation of session
from sqlalchemy.orm import sessionmaker
session = sessionmaker(bind= engine)

session = session()

In [22]:
s1 = Student(name = "Ram", age= 20, address = "pokhara" )
s2 = Student(name = "Shyam", age= 21, address = "kathmandu" )
s3 = Student(name = "Hari", age= 22, address = "butwal" )
session.add(s1)
session.add(s2)
session.add(s3)
session.commit()


2025-12-31 13:18:53,690 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-31 13:18:53,694 INFO sqlalchemy.engine.Engine INSERT INTO students (name, age, address) VALUES (?, ?, ?) RETURNING id
2025-12-31 13:18:53,696 INFO sqlalchemy.engine.Engine [generated in 0.00022s (insertmanyvalues) 1/6 (ordered; batch not supported)] ('Ram', 20, 'pokhara')
2025-12-31 13:18:53,698 INFO sqlalchemy.engine.Engine INSERT INTO students (name, age, address) VALUES (?, ?, ?) RETURNING id
2025-12-31 13:18:53,699 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/6 (ordered; batch not supported)] ('Shyam', 21, 'kathmandu')
2025-12-31 13:18:53,700 INFO sqlalchemy.engine.Engine INSERT INTO students (name, age, address) VALUES (?, ?, ?) RETURNING id
2025-12-31 13:18:53,701 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/6 (ordered; batch not supported)] ('Hari', 22, 'butwal')
2025-12-31 13:18:53,701 INFO sqlalchemy.engine.Engine INSERT INTO students (name, age, address) VALUES (?, ?, ?) RETURNING id
2025

In [23]:
data = session.query(Student).all()
for row in data:
    print(row.id, row.name, row.age, row.address)

2025-12-31 13:21:36,720 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-31 13:21:36,725 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.age AS students_age, students.address AS students_address 
FROM students
2025-12-31 13:21:36,726 INFO sqlalchemy.engine.Engine [generated in 0.00142s] ()
1 Ram 20 pokhara
2 Shyam 21 kathmandu
3 Hari 22 butwal
4 Ram 20 pokhara
5 Shyam 21 kathmandu
6 Hari 22 butwal


In [28]:
address_itahari = session.query(Student).filter_by(address = "kathmandu")
for rows in address_itahari:
    print(row.id, row.name, row.age, row.address)

2025-12-31 13:24:33,345 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.age AS students_age, students.address AS students_address 
FROM students 
WHERE students.address = ?
2025-12-31 13:24:33,347 INFO sqlalchemy.engine.Engine [cached since 32.76s ago] ('kathmandu',)
6 Hari 22 butwal
6 Hari 22 butwal


# sir le sikako 2tai tarika use garera solve gara -> 
create a book table
# difference betwee RDBMS and DBMS

In [ ]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["school"]            #database
collection = db["students"]          #collection(like a table)